<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ym7i1zjc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ym7i1zjc
  Resolved https://github.com/huggingface/transformers to commit 638d49983f36af910934b38771b4e55c835c1774
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [17]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [18]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="test")
#column_name = "audio, normalized_text"

In [19]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("tiny")

In [20]:
model = whisper.load_model("tiny")

100%|█████████████████████████████████████| 72.1M/72.1M [00:00<00:00, 92.2MiB/s]


In [21]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [22]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr")
    return result["text"]

In [23]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [24]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [25]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)


 # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1
    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.8214285714285714, Precision: 0.3157894736842105, Recall: 0.36, F1: 0.3364485981308411
Processed 2 files, WER: 0.9411764705882353, Precision: 0.25806451612903225, Recall: 0.24242424242424243, F1: 0.25
Processed 3 files, WER: 0.5862068965517241, Precision: 0.36, Recall: 0.375, F1: 0.3673469387755102
Processed 4 files, WER: 0.9142857142857143, Precision: 0.15625, Recall: 0.1724137931034483, F1: 0.1639344262295082
Processed 5 files, WER: 0.8333333333333334, Precision: 0.29411764705882354, Recall: 0.2777777777777778, F1: 0.28571428571428575
Processed 6 files, WER: 0.8043478260869565, Precision: 0.325, Recall: 0.3170731707317073, F1: 0.3209876543209877
Processed 7 files, WER: 0.7894736842105263, Precision: 0.2777777777777778, Recall: 0.2777777777777778, F1: 0.2777777777777778
Processed 8 files, WER: 0.925, Precision: 0.22580645161290322, Recall: 0.18421052631578946, F1: 0.20289855072463767
Processed 9 files, WER: 0.7272727272727273, Precision: 0.36666666666666664, R